# Using Bitquery GraphQL API

In [ ]:
API_key = "BQYqISAGCYfoDCPjnoX7BmQZiw3qABkn"

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests


def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'YOUR API KEY'}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code,
                        query))


# The GraphQL query

query = """
query{
  bitcoin{
    blocks{
      count
    }
   }
}
"""
result = run_query(query)  # Execute the query
print 'Result - {}'.format(result)

In [ ]:
# total users created (number of associated accounts created): COUNT TRANSACTION WHERE ALL OF BELOW HAPPED IN ONE tx

    ## associated token program [ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL]: CREATE
        # MINT: AFbX8oGjGpmVFywbVouvhQSRmiW2aR1mohfahi4Y2AdB
        # ACCOUNT: NOT HLS5Y68QSQgJP7wUbbbbCjEnMknVZrHXYDwwVaDcsdK7
        # WALLET: NOT STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK
        
        ## system program: CREATE ACCOUNT
            # FROM ADDRESS: NOT STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK
            
        ## token program: INITIALIZE ACCOUNT
            # MINT: AFbX8oGjGpmVFywbVouvhQSRmiW2aR1mohfahi4Y2AdB            
            
    ## TOKEN PROGRAM [TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA]: TRANSFER (CHECKED)
        # AUTHORITY: STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK
        # MINT: AFbX8oGjGpmVFywbVouvhQSRmiW2aR1mohfahi4Y2AdB
        # SOURCE: HLS5Y68QSQgJP7wUbbbbCjEnMknVZrHXYDwwVaDcsdK7
    

# total GST transfered from STPEN auth

    ## TOKEN PROGRAM [TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA]: TRANSFER (CHECKED)
        # AUTHORITY: STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK
        # MINT: AFbX8oGjGpmVFywbVouvhQSRmiW2aR1mohfahi4Y2AdB
        # SOURCE: HLS5Y68QSQgJP7wUbbbbCjEnMknVZrHXYDwwVaDcsdK7

        
# total GST traansfered back to STEPN, i.e. fees (to auth addres??)

     



# total burned





# total in treasury

# 

# Using solana.py

In [3]:
# prep
import solana, json, mitosheet, pprint, json
from datetime import datetime
import pandas as pd
from solana.rpc.api import Client

mint_address = 'AFbX8oGjGpmVFywbVouvhQSRmiW2aR1mohfahi4Y2AdB'  # GST mint
auth_address = 'STEPNq2UGeGSzCyGVr2nMQAzf8xuejwqebd84wcksCK'  # STEPN wallet

#endpoint = 'https://api.devnet.solana.com'    # probably for `developing`
#endpoint = 'https://api.testnet.solana.com'   # probably for `testing`
#endpoint = 'https://api.mainnet-beta.solana.com'
# endpoint = 'https://solana-api.projectserum.com'
endpoint = 'https://cool-thrumming-bush.solana-mainnet.quiknode.pro/e3acee1ce0fcfc581bd2d2178784f785de1ecaf4/'

solana_client = Client(endpoint)

In [4]:
# get list of tx signatures (latest 1000)
resp = solana_client.get_signatures_for_address(mint_address) #, limit=1, before='89Tv9s2uMGaoxB8ZF1LV9nGa72GQ9RbkeyCDvfPviWesZ6ajZBFeHsTPfgwjGEnH7mpZa7jQBXAqjAfMrPirHt2')
    

In [7]:
if 'result' in resp:
    print('len:', len(resp['result']))        
else:        
    print('no result returned')
    
df_sigs = pd.DataFrame(resp['result'])
df_sigs = df_sigs.loc[df_sigs['confirmationStatus']=='finalized', :]

df_sigs.tail(1)

len: 1000


,blockTime,confirmationStatus,err,memo,signature,slot
999,1654765412,finalized,None,None,481VUkjRMWe1VDs3VZPJSTmLSgzrHaxmAMuofzVbv4oAE8ESTWYt51oHSMSf4sLdobXr4QSLFNGxzR1eHPHtBux2,136896105


In [38]:
# get first and last timestamp
print('last timestamp: ', datetime.fromtimestamp(df_sigs['blockTime'].iloc[0]))
print('first timestamp: ', datetime.fromtimestamp(df_sigs['blockTime'].iloc[-1]))

last timestamp:  2022-06-07 01:44:33
first timestamp:  2022-06-07 00:41:40


In [ ]:
# loop through timestamps
pp = pprint.PrettyPrinter(indent=4)
counter = 0
for tstamp in df_sigs['signature']:      
    tx = solana_client.get_transaction(tstamp)
    # rows2see = ['innerInstructions', 'logMessages', 'postTokenBalances', 'message', 'signatures']
    # cols2see = ['meta', 'transaction']
    # record = pd.DataFrame(tx['result']).loc[rows2see, cols2see]
    print('\ncount: ', counter)
    print('signature: ', tx['result']['transaction']['signatures'])
    pprint.pprint(tx['result']['meta'])
    counter += 1
    # if tstamp == df_sigs['signature'][3]: break

In [95]:
solana_client.get_token_accounts_by_owner('64wNpDM4DHwAUpneu7gofNAXrXxzv5cvmcFYQomy1UyT', ')

NameError: name 'mint' is not defined

# get transfer amount

In [13]:
post = resp['result']['meta']['postTokenBalances']
pre = resp['result']['meta']['preTokenBalances']

before = pd.DataFrame(pre).drop(columns='uiTokenAmount').set_index('accountIndex')
before_token = pd.DataFrame(
    [{**{'accountIndex':x['accountIndex']}, **x['uiTokenAmount']} for x in pre \
     if (x['mint'] == mint_address) & (x['owner'] != auth_address)]
     ).set_index('accountIndex').drop(columns=['uiAmount', 'uiAmountString'])
a1 = pd.concat([before, before_token], axis=1, join='inner')
a1['amount'] = pd.to_numeric(a1['amount'])

after = pd.DataFrame(post).drop(columns='uiTokenAmount').set_index('accountIndex')
after_token = pd.DataFrame(
    [{**{'accountIndex':x['accountIndex']}, **x['uiTokenAmount']} for x in post \
     if (x['mint'] == mint_address) & (x['owner'] != auth_address)]
     ).set_index('accountIndex').drop(columns=['uiAmount', 'uiAmountString'])
a2 = pd.concat([after, after_token], axis=1, join='inner')
a2['amount'] = pd.to_numeric(a2['amount'])

a3 = a2.loc[~a2.index.isin(a1.index), :]
a4 = a2.loc[a2.index.isin(a1.index), :]
assert(a4[['mint', 'owner', 'decimals']].equals(a1[['mint', 'owner', 'decimals']]))

a6 = pd.concat([a4.drop(columns=['amount']), a4[['amount']]-a1[['amount']]], axis=1)

a7 = pd.concat([a3, a6])

a7

{'blockHeight': None,
 'blockTime': None,
 'blockhash': 'DdwYftNVqFsFUAfSqD59nT356KeSDfVJkUHjsGdvjR8y',
 'parentSlot': 99,
 'previousBlockhash': 'AXu3HSYxcaQZbBsZJM98NiSWrQoHvunZmC2i5KF3pk1G',
 'rewards': [],
 'transactions': [{'meta': None,
   'transaction': {'message': {'accountKeys': ['CakcnaRDHka2gXyfbEd2d3xsvkJkqsLw2akB3zsN1D2S',
      '9bRDrYShoQ77MZKYTMoAsoCkU7dAR24mxYCBjXLpfEJx',
      'SysvarS1otHashes111111111111111111111111111',
      'SysvarC1ock11111111111111111111111111111111',
      'Vote111111111111111111111111111111111111111'],
     'header': {'numReadonlySignedAccounts': 0,
      'numReadonlyUnsignedAccounts': 3,
      'numRequiredSignatures': 1},
     'instructions': [{'accounts': [1, 2, 3, 0],
       'data': '37u9WtQpcm6ULa3W9Axfm2UGrJFgLgF5yHTAoZnPTBbcZ4m7NwQdwRCVWcXhoGvaGYQK4CFH',
       'programIdIndex': 4}],
     'recentBlockhash': 'AXu3HSYxcaQZbBsZJM98NiSWrQoHvunZmC2i5KF3pk1G'},
    'signatures': ['2Joi4qeU7e7N79DRYRGfvWmTaqBDdFsCowNaCrKaBHVmRpmQBqizPZPmZCcfteB

In [12]:
solana_client.get_block(100)

{'jsonrpc': '2.0',
 'result': {'blockHeight': None,
  'blockTime': None,
  'blockhash': 'DdwYftNVqFsFUAfSqD59nT356KeSDfVJkUHjsGdvjR8y',
  'parentSlot': 99,
  'previousBlockhash': 'AXu3HSYxcaQZbBsZJM98NiSWrQoHvunZmC2i5KF3pk1G',
  'rewards': [],
  'transactions': [{'meta': None,
    'transaction': {'message': {'accountKeys': ['CakcnaRDHka2gXyfbEd2d3xsvkJkqsLw2akB3zsN1D2S',
       '9bRDrYShoQ77MZKYTMoAsoCkU7dAR24mxYCBjXLpfEJx',
       'SysvarS1otHashes111111111111111111111111111',
       'SysvarC1ock11111111111111111111111111111111',
       'Vote111111111111111111111111111111111111111'],
      'header': {'numReadonlySignedAccounts': 0,
       'numReadonlyUnsignedAccounts': 3,
       'numRequiredSignatures': 1},
      'instructions': [{'accounts': [1, 2, 3, 0],
        'data': '37u9WtQpcm6ULa3W9Axfm2UGrJFgLgF5yHTAoZnPTBbcZ4m7NwQdwRCVWcXhoGvaGYQK4CFH',
        'programIdIndex': 4}],
      'recentBlockhash': 'AXu3HSYxcaQZbBsZJM98NiSWrQoHvunZmC2i5KF3pk1G'},
     'signatures': ['2Joi4qeU7e7N79

In [8]:
dir(solana_client)

['_Client__get_token_accounts',
 '_Client__post_send_with_confirm',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_before_rpc_config_key',
 '_comm_key',
 '_commitment',
 '_data_slice_key',
 '_encoding_key',
 '_get_account_info_args',
 '_get_balance_args',
 '_get_block_args',
 '_get_block_commitment_args',
 '_get_block_height_args',
 '_get_block_time_args',
 '_get_blocks_args',
 '_get_cluster_nodes',
 '_get_confirmed_block_args',
 '_get_confirmed_blocks_args',
 '_get_confirmed_signature_for_address2_args',
 '_get_confirmed_transaction_args',
 '_get_epoch_info_args',
 '_get_epoch_schedule',
 '_get_fee_calculator_for_blockhash_args',
 '_get_fee_rate_governor',
 '_get_f